# Utility functions

In [47]:
import os
from math import sqrt
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from scipy.signal import medfilt
import sys
sys.path.append(os.path.join(os.getcwd(), 'common'))
import utility
import random
import pywt
from skimage.transform import rescale

In [48]:
def wpsnr(img1, img2):
    img1 = np.float32(img1)/255.0
    img2 = np.float32(img2)/255.0
    difference = img1-img2
    same = not np.any(difference)
    if same is True:
        return 9999999
    w = np.genfromtxt('csf.csv', delimiter=',')
    ew = convolve2d(difference, np.rot90(w,2), mode='valid')
    decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
    return decibels

## Edges detection

In [49]:
def sobel_edges(img):
  sobelimage=img.copy()
  sobelx = cv2.Sobel(sobelimage,cv2.CV_64F,1,0,ksize=3)
  sobely = cv2.Sobel(sobelimage,cv2.CV_64F,0,1,ksize=3)
  magnitude = np.sqrt(sobelx**2 + sobely**2)
  magnitude = cv2.convertScaleAbs(magnitude)
  _, thresholded = cv2.threshold(magnitude, 100, 255, cv2.THRESH_BINARY)
  return thresholded
  
def canny_edges(img):
  th1=30
  th2=60
  d=2
  edgeresult=img.copy()
  edgeresult = cv2.GaussianBlur(edgeresult, (2*d+1, 2*d+1), -1)[d:-d,d:-d]
  edgeresult = edgeresult.astype(np.uint8)
  edges = cv2.Canny(edgeresult, th1, th2)
  return edges

## Basic attack functions

In [71]:
def blur(img, sigma):
  attacked = gaussian_filter(img, sigma)
  return attacked

def median(img, sigma):
  attacked = medfilt(img, sigma)
  return attacked

def awgn(img, std, seed):
  mean = 0.0
  np.random.seed(seed)
  attacked = img + np.random.normal(mean, std, img.shape)
  attacked = np.clip(attacked, 0, 255)
  return attacked

def jpeg_compression(img, qf):
  img = Image.fromarray(img)
  img = img.convert('L') 
  img.save('tmp.jpg',"JPEG", quality=qf)
  attacked = Image.open('tmp.jpg')
  attacked = np.asarray(attacked,dtype=np.uint8)
  os.remove('tmp.jpg')
  return attacked

def resizing(img, scale):
  x, y = img.shape
  attacked = rescale(img, scale, anti_aliasing=True, mode='reflect')
  attacked = rescale(attacked, 1/scale, anti_aliasing=True, mode='reflect')
  attacked = np.asarray(attacked * 255, dtype=np.uint8)
  attacked = cv2.resize(attacked, (y, x), interpolation=cv2.INTER_LINEAR)
  return attacked

# Edge-preserving blur

In [51]:
def edge_preserving_blur(img, sigma, edge_func, blur_func):
  edges = edge_func(img)
  if edges.max() > 0:
    edges = edges / edges.max()
  edges = cv2.resize(edges, (img.shape[1], img.shape[0]))
  blurred_img = blur_func(img, sigma)
  attacked = edges * img + (1 - edges) * blurred_img
  return (attacked, 'Edge-preserving blur', '')

In [52]:
def edge_blur(img, sigma, edge_func, blur_func):
    edges = edge_func(img)
    if edges.max() > 0:
        edges = edges / edges.max()
    edges = cv2.resize(edges, (img.shape[1], img.shape[0]))
    blurred_img = blur_func(img, sigma)
    attacked = (1 - edges) * img + edges * blurred_img
    return (attacked, 'Blur edge ', '')

# AWGN on edges

In [53]:
def edge_awgn(img, std, seed, edge_func):
    attacked = img.copy()
    edges_results = img.copy()
    global_awgn = awgn(img, std, seed)
    edges = edge_func(img)
    edges = edges / edges.max()
    edges = cv2.resize(edges, (img.shape[1], img.shape[0]))
    edges_results[edges > 0] = [255]
    attacked[edges > 0] = global_awgn[edges > 0] 
    return (attacked, 'Awgn on edges', '')

# Blur on DWT

In [54]:
def blur_dwt(img, blur_func, sigma):
    coeffs2 = pywt.dwt2(img, 'haar')
    LL, (LH, HL, HH) = coeffs2

    LL_blurred = blur_func(LL, sigma)
    LH_blurred = blur_func(LH, sigma)
    HL_blurred = blur_func(HL, sigma)
    HH_blurred = blur_func(HH, sigma)
    
    #coeffs2_blurred = (LL_blurred, (LH_blurred, HL_blurred, HH_blurred))
    coeffs2_blurred = (LL_blurred, (LH, HL, HH))
    reconstructed_image = pywt.idwt2(coeffs2_blurred, 'haar')
    reconstructed_image = np.clip(reconstructed_image, 0, 255)
    return (reconstructed_image.astype(np.uint8),'Blur on dwt','')

In [55]:
def blur_dwt(img, blur_func, sigma):
    coeffs2 = pywt.dwt2(img, 'haar')
    LL, (LH, HL, HH) = coeffs2

    LH_blurred = blur_func(LH, sigma)
    HL_blurred = blur_func(HL, sigma)
    HH_blurred = blur_func(HH, sigma)
    
    coeffs3 = pywt.dwt2(LL, 'haar')
    LL2, (LH2, HL2, HH2) = coeffs3

    LH2_blurred = blur_func(LH2, sigma)
    HL2_blurred = blur_func(HL2, sigma)
    HH2_blurred = blur_func(HH2, sigma)
    
    coeffs3_blurred = (LL2, (LH2_blurred, HL2_blurred, HH2_blurred))
    reconstructed_ll = pywt.idwt2(coeffs3_blurred, 'haar')
    reconstructed_ll = np.clip(reconstructed_ll, 0, 255)
    
    coeffs2_blurred = (reconstructed_ll, (LH_blurred, HL_blurred, HH_blurred))
    reconstructed_image = pywt.idwt2(coeffs2_blurred, 'haar')
    reconstructed_image = np.clip(reconstructed_image, 0, 255)
    return (reconstructed_image.astype(np.uint8),'Blur on dwt','')

# Plot images and check wpsnr

In [56]:
original = cv2.imread('lena_grey.bmp', 0)

attack1 = jpeg_compression(original, 10)
attack2 = edge_preserving_blur(attack1.copy(), [3,3], sobel_edges, blur)[0]
#attack3 = blur_dwt(attack1.copy(), sobel_edges, median, [7,7], 0, None, None, None)[0]
attack3 = blur_dwt(attack1.copy(), median, [3,3])[0]
attack4 = resizing(attack1.copy(), 0.95)

utility.visualize_images_with_desc([attack1, attack2, attack3, attack4],[str(wpsnr(original, attack1)), str(wpsnr(original, attack2)), str(wpsnr(original, attack3)), str(wpsnr(original, attack4))])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

# Possible attacks

In [67]:
attack_list = [
    lambda img,k: edge_preserving_blur(img, [k,k], sobel_edges, blur),
    lambda img,k: edge_preserving_blur(img, [k,k], sobel_edges, median),
    lambda img,k: edge_preserving_blur(img, [k,k], canny_edges, blur),
    lambda img,k: edge_preserving_blur(img, [k,k], canny_edges, median),
    
    lambda img,k: edge_blur(img, [k,k], sobel_edges, blur),
    lambda img,k: edge_blur(img, [k,k], sobel_edges, median),
    lambda img,k: edge_blur(img, [k,k], canny_edges, blur),
    lambda img,k: edge_blur(img, [k,k], canny_edges, median),
]
attack_awgn = [
    lambda img,std: edge_awgn(img,std, random.randint(10,100), sobel_edges),
    lambda img,std: edge_awgn(img,std, random.randint(10,100), canny_edges),
    
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], sobel_edges, blur)[0], std, random.randint(10,100), sobel_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], sobel_edges, median)[0], std, random.randint(10,100), sobel_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], canny_edges, blur)[0],std, random.randint(10,100), sobel_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], canny_edges, median)[0], std, random.randint(10,100), sobel_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], sobel_edges, blur)[0], std, random.randint(10,100), canny_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], sobel_edges, median)[0],std, random.randint(10,100), canny_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], canny_edges, blur)[0], std, random.randint(10,100), canny_edges),
    lambda img,std: edge_awgn(edge_preserving_blur(img, [3,3], canny_edges, median)[0], std, random.randint(10,100), canny_edges),
]

blur_dwt_attack = [
    lambda img,k: blur_dwt(img, blur, [k,k]),
    lambda img,k: blur_dwt(img, median, [k,k]),
]

resizing_compressed_attack = [
    lambda img, qf, s: resizing(jpeg_compression(img, int(100-qf)), float(1-s)),
]

# Embedding

In [65]:
import pywt
import os
from scipy.fft import dct, idct
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import convolve2d
from math import sqrt
import sys
sys.path.append(os.path.join(os.getcwd(), 'common'))
import detection
import utility
import attacks
from tqdm import tqdm
import embedding
import importlib
importlib.reload(utility)
importlib.reload(detection)
importlib.reload(attacks)
importlib.reload(embedding)


# def awgn(img, std, seed):
#   mean = 0.0   # some constant
#   np.random.seed(seed)
#   attacked = img + np.random.normal(mean, std, img.shape)
#   attacked = np.clip(attacked, 0, 255)
#   return np.uint8(attacked)

def wpsnr(img1, img2):
  img1 = np.float32(img1)/255.0
  img2 = np.float32(img2)/255.0
  difference = img1-img2
  same = not np.any(difference)
  if same is True:
      return 9999999
  w = np.genfromtxt('csf.csv', delimiter=',')
  ew = convolve2d(difference, np.rot90(w,2), mode='valid')
  decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
  return decibels





img_path='lena_grey.bmp'
N = 1024

#v = 'additive'
#alpha = 24 
v = 'multiplicative'
alpha = 0.5
np.random.seed(seed=123)


image = cv2.imread(img_path, 0)
mark = np.load('ammhackati.npy')
watermarked1 =  embedding.embedding(image,mark,  alpha)



In [24]:
attack_functions = attack_list
progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
total = 0
succ = 0
for attack_fn in progress_bar:
    for k in [1,3,5,7,9,11]:
        attacked,attack_name,usd = attack_fn(watermarked1,k)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected,"k":k})
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
print("Average wpsnr of succesfull attacks: ", (total/succ))    
for h in history:
    print(h) 

Applying attacks: 100%|██████████| 8/8 [01:04<00:00,  8.12s/it]

Average wpsnr of succesfull attacks:  36.46699845271097
{'attack': 'Edge-preserving blur', 'wpsnr': 49.63333872193943, 'param': '', 'detection': 1, 'k': 1}
{'attack': 'Edge-preserving blur', 'wpsnr': 36.33351646426558, 'param': '', 'detection': 1, 'k': 3}
{'attack': 'Edge-preserving blur', 'wpsnr': 32.40785918147759, 'param': '', 'detection': 1, 'k': 5}
{'attack': 'Edge-preserving blur', 'wpsnr': 30.36215446577994, 'param': '', 'detection': 1, 'k': 7}
{'attack': 'Edge-preserving blur', 'wpsnr': 29.020954738432366, 'param': '', 'detection': 1, 'k': 9}
{'attack': 'Edge-preserving blur', 'wpsnr': 28.033833525260327, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Edge-preserving blur', 'wpsnr': 9999999, 'param': '', 'detection': 1, 'k': 1}
{'attack': 'Edge-preserving blur', 'wpsnr': 52.31747387408531, 'param': '', 'detection': 1, 'k': 3}
{'attack': 'Edge-preserving blur', 'wpsnr': 46.638601125876455, 'param': '', 'detection': 1, 'k': 5}
{'attack': 'Edge-preserving blur', 'wpsnr': 42.730

In [31]:
attack_functions = attack_awgn
progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
total = 0
succ = 0
for attack_fn in progress_bar:
    for std in np.arange(0.0,5.0,0.5):
        attacked,attack_name,usd = attack_fn(watermarked1,std)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected,"std":std})
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
print("Average wpsnr of succesfull attacks: ", (total/succ))    
for h in history:
    print(h) 

Applying attacks: 100%|██████████| 10/10 [01:59<00:00, 11.93s/it]

Average wpsnr of succesfull attacks:  33.718050296175974
{'attack': 'Awgn on edges', 'wpsnr': 69.8672577458093, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 68.8323057699278, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 67.44584456648306, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 65.74582777557777, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 64.19406604772436, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 62.69906260540437, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 61.602084431255626, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 60.29563139615968, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 59.54332376111991, 'param': '', 'detection': 1, 'k': 11}
{'attack': 'Awgn on edges', 'wpsnr': 58.508335906560205, 'param': '', 'detection': 1, 'k': 11}
{'a

In [12]:
attack_functions = blur_dwt_attack
progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
total = 0
succ = 0
for attack_fn in progress_bar:
    for k in [1,3,5,7,9,11]:
        attacked,attack_name,usd = attack_fn(watermarked1, k)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected})
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            
if succ > 0 :
    print("Average wpsnr of succesfull attacks: ", (total/succ))    
else:
     print("Sto attacco fa schifo")
for h in history:
    print(h) 

Applying attacks:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Utente\Documents\Università\Master\MultimediaDataSecurity\CTM\multimedia-data-security\common\detection.py:91: RuntimeWarning: invalid value encountered in scalar divide
  s = np.sum(np.multiply(X, X_star)) / (np.sqrt(np.sum(np.multiply(X, X))) * np.sqrt(np.sum(np.multiply(X_star, X_star))))
Applying attacks: 100%|██████████| 2/2 [00:26<00:00, 13.02s/it]

Sto attacco fa schido
{'attack': 'Blur on dwt', 'wpsnr': 49.11787239753452, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.28465078733278, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.23718252633944, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.22670429269232, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.226138992707654, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.223192312805146, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 64.35450768674929, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 48.64433126937609, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.29607449238821, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.311031250875224, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.305526054702874, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 47.283223487215466, 'param

In [25]:
attack_functions = blur_dwt_attack
progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
total = 0
succ = 0
for attack_fn in progress_bar:
    for k in [1,3,5,7,9,11]:
        attacked,attack_name,usd = attack_fn(watermarked1, k)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected})
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            
if succ > 0 :
    print("Average wpsnr of succesfull attacks: ", (total/succ))    
else:
     print("Sto attacco fa schifo")
for h in history:
    print(h) 

Applying attacks:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Utente\Documents\Università\Master\MultimediaDataSecurity\CTM\multimedia-data-security\common\detection.py:91: RuntimeWarning: invalid value encountered in scalar divide
  s = np.sum(np.multiply(X, X_star)) / (np.sqrt(np.sum(np.multiply(X, X))) * np.sqrt(np.sum(np.multiply(X_star, X_star))))
Applying attacks: 100%|██████████| 2/2 [00:27<00:00, 13.78s/it]

Average wpsnr of succesfull attacks:  23.877153087678266
{'attack': 'Blur on dwt', 'wpsnr': 23.955675441693444, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.868186140063344, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.857790618707003, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.854031700027484, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.852434428168575, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.851178754990432, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 24.16215958817498, 'param': '', 'detection': 1}
{'attack': 'Blur on dwt', 'wpsnr': 23.925789256820487, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.88129048313516, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.87075384340949, 'param': '', 'detection': 0}
{'attack': 'Blur on dwt', 'wpsnr': 23.866849432327058, 'param': '', 'detection': 0}
{'attack': 'Blur on dw

In [77]:
# attack_functions = resizing_compressed_attack
# progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
total = 0
succ = 0
#for attack_fn in progress_bar:
for qf in range(0, 100, 10):
    #attacked,attack_name,usd = attack_fn(watermarked1, qf, s)
    attacked = jpeg_compression(watermarked1, (100-qf))
    #attack_fn(watermarked1, qf, s)
    attack_name="resizing of compressend image"
    #used = ""
    usd = "qf=" + str(100-qf)
    detected = detection.detection(image, watermarked1,attacked, alpha, 2)
    wpsnr_attacked = wpsnr(watermarked1, attacked)
    history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected})
    if detected == 0:
        total += wpsnr_attacked
        succ +=1
            
if succ > 0 :
    print("Average wpsnr of succesfull attacks: ", (total/succ))    
else:
     print("Sto attacco fa schifo")
for h in history:
    print(h) 

Average wpsnr of succesfull attacks:  40.33726188560973
{'attack': 'resizing of compressend image', 'wpsnr': 63.89619641241409, 'param': 'qf=100 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 61.59373590122339, 'param': 'qf=90 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 58.19675814859977, 'param': 'qf=80 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 55.48935216145725, 'param': 'qf=70 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 53.50319270762843, 'param': 'qf=60 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 51.93653844449672, 'param': 'qf=50 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 50.17962110712563, 'param': 'qf=40 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 48.1673613039452, 'param': 'qf=30 s=0.5', 'detection': 1}
{'attack': 'resizing of compressend image', 'wps

In [79]:
# attack_functions = resizing_compressed_attack
# progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
max = 0
total = 0
succ = 0
#for attack_fn in progress_bar:
for qf in range(0, 100, 10):
    for s in np.arange(0, 1, 0.1):
        #attacked,attack_name,usd = attack_fn(watermarked1, qf, s)
        attacked = resizing(jpeg_compression(watermarked1, (100-qf)), (1-s))
        #attack_fn(watermarked1, qf, s)
        attack_name="resizing of compressend image"
        #used = ""
        usd = "qf=" + str(100-qf) + " s=" + str(1-s)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd, "detection":detected})
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            
if succ > 0 :
    print("Average wpsnr of succesfull attacks: ", (total/succ))  
    print("Max wpsnr: ", max)  
else:
     print("Sto attacco fa schifo")
for h in history:
    print(h) 

c:\Users\Utente\Documents\Università\Master\MultimediaDataSecurity\CTM\multimedia-data-security\common\detection.py:91: RuntimeWarning: invalid value encountered in scalar divide
  s = np.sum(np.multiply(X, X_star)) / (np.sqrt(np.sum(np.multiply(X, X))) * np.sqrt(np.sum(np.multiply(X_star, X_star))))


Average wpsnr of succesfull attacks:  37.58054723724023
Max wpsnr:  45.54235181220024
{'attack': 'resizing of compressend image', 'wpsnr': 61.488370912341715, 'param': 'qf=100 s=1.0', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 49.388259270983355, 'param': 'qf=100 s=0.9', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 48.40221302717825, 'param': 'qf=100 s=0.8', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 45.358514754766254, 'param': 'qf=100 s=0.7', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 45.69943112962917, 'param': 'qf=100 s=0.6', 'detection': 1}
{'attack': 'resizing of compressend image', 'wpsnr': 41.91797049742597, 'param': 'qf=100 s=0.5', 'detection': 0}
{'attack': 'resizing of compressend image', 'wpsnr': 39.0232329250087, 'param': 'qf=100 s=0.3999999999999999', 'detection': 0}
{'attack': 'resizing of compressend image', 'wpsnr': 35.81593234538219, 'param': 'qf=100 s=0.2999999999999